# NYC Yellow Taxi Trips

## Sobre o Dataset

**Fonte dos dados:** Kaggle - NYC Yellow Taxi Trip Data (elemento) - Version 2
**Cobertura:** 1º trimestre de 2016
**Volume original:** ~ 5,5GB
**Formato:** CSV com múltiplos arquivos mensais separados

Cada linha representa uma corrida individual de taxi, contendo informações de data e hora de embarque e desembarque, localização de origem e de destino, número de passageiros, distância percorrida, tarifas, gorjetas e forma de pagamento.

## Amostragem

Cada arquivo CSV possui aproximadamente 1,8 GB, o volume de dados é inviável para carregamento integral em um ambiente com 8GB de RAM.

O dataset completo do 1º trimestre de 2016 contém 34.499.859 corridas:
- Janeiro: 10.906.858 corridas
- Fevereiro: 11.382.049 corridas
- Março: 12.210.952 corridas

Por isso, foi adotada uma amostragem aleatória por arquivo mensal, no qual o total de linhas foi contabilizado e 500.000 registros foram selecionados aleatoriamente via numpy, totalizando 1,5 milhão de registros na amostra de trabalho.

Essa abordagem é estatisticamente mais robusta do que simplesmente carregar as primeiras N linhas, pois garante representatividade ao longo de todo o período mensal, evitando viés.

## Perguntas respondidas
- Qual é a evolução do volume de corridas e faturamento ao longo do trimestre?
- Quais são os horários e dias de maior demanda?
- Quais regiões geram mais receita?
- O que influencia o valor da gorjeta?
- Quais padrões indicam corridas atípicas ou suspeitas?

## Tecnologias utilizadas
- Python 3
- Pandas
- NumPy
- Matplotlib
- Plotly
- VS Code + Jupyter Notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os

sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = (13, 5)
plt.rcParams['font.size'] = 12

In [5]:
import subprocess

AMOSTRA_POR_ARQUIVO = 500_000

pasta = '../data/'
arquivos_csv = sorted([
    f for f in os.listdir(pasta) 
    if f.startswith('yellow_tripdata_2016') 
    and f.endswith('.csv')
])

dfs = []
for arquivo in arquivos_csv:

    # contar total de linhas no arquivo
    resultado = subprocess.run(
        ['wc', '-l', pasta + arquivo], 
        capture_output=True, text=True
    )
    total_linhas = int(resultado.stdout.strip().split()[0]) - 1  # Subtrair 1 para o cabeçalho
    print(f"{arquivo} — {total_linhas:,} linhas")

    # calcular quais linhas pular para amostra aleatória
    linhas_pular = sorted(
        np.random.choice(
            range(1, total_linhas + 1),  # Linhas começam em 1 por causa do cabeçalho
            size=total_linhas - AMOSTRA_POR_ARQUIVO,
            replace=False
        )
    )
    
    # carregar apenas as linhas selecionadas
    df_temp = pd.read_csv(
        pasta + arquivo, 
        skiprows=linhas_pular,
        low_memory=False
    )
    df_temp['mes_origem'] = arquivo.split('-')[1].replace('.csv', '')
    dfs.append(df_temp)
    print(f"Amostra carregada - {len(df_temp):,} linhas\n")

yellow_tripdata_2016-01.csv — 10,906,858 linhas
Amostra carregada - 500,000 linhas

yellow_tripdata_2016-02.csv — 11,382,049 linhas
Amostra carregada - 500,000 linhas

yellow_tripdata_2016-03.csv — 12,210,952 linhas
Amostra carregada - 500,000 linhas



## Carregamento dos dados

Nesta etapa são carregados os 3 arquivos CSV mensais, unificados em um único DataFrame, e é feito o reconhecimento inicial do dataset.

In [6]:
df = pd.concat(dfs, ignore_index=True)
print(f"DataFrame final: {len(df):,} linhas")

df.head()

DataFrame final: 1,500,000 linhas


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,mes_origem
0,2,2016-01-01 00:00:00,2016-01-01 00:26:45,2,7.45,-73.994057,40.719990,1,N,-73.966362,40.789871,2,26.0,0.5,0.5,0.0,0.0,0.3,27.3,01
1,1,2016-01-01 00:00:09,2016-01-01 00:07:18,2,1.20,-73.963913,40.712173,1,N,-73.951332,40.712200,2,7.0,0.5,0.5,0.0,0.0,0.3,8.3,01
2,2,2016-01-01 00:00:18,2016-01-01 00:05:44,5,1.17,-73.963058,40.775017,1,N,-73.951744,40.778801,2,6.0,0.5,0.5,0.0,0.0,0.3,7.3,01
3,1,2016-01-01 00:00:52,2016-01-01 00:08:00,3,1.80,-73.983871,40.742409,1,N,-74.002365,40.726349,2,8.0,0.5,0.5,0.0,0.0,0.3,9.3,01
4,1,2016-01-01 00:01:13,2016-01-01 00:22:09,1,2.50,-73.994255,40.731995,1,N,-73.978638,40.751389,2,14.5,0.5,0.5,0.0,0.0,0.3,15.8,01


## Data Profiling

**1. Estrutura e tipos de dados (df.info())**
Revela quantas colunas existem, o tipo de dado de cada uma e se há valores nulos. É nessa etapa que identificamos colunas que precisarão de conversão de tipo e colunas com dados pendentes que exigirão tratamento.

**2. Valores nulos**
Colunas com valores ausentes precisam de uma decisão: preencher com a mediana, com a moda, com um valor padrão, ou remover os registros. Cada decisão tem impacto direto na qualidade da análise e deve ser documentada e justificada.

**3. Estatísticas descritivas (df.describe())**
Revela a distribuição de cada coluna numérica (mínimo, máximo, média e percentis). É aqui que são identificados valores impossíveis ou suspeitos, como tarifas negativas, distâncias zero e corridas sem passageiros, que serão tratados na etapa de limpeza.

**4. Valores únicos nas colunas categóricas**
Revela quantas categorias distintas existem em cada coluna de texto ou código, permitindo identificar inconsistências como categorias inválidas, erros de digitação ou códigos não documentados.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1500000 non-null  int64  
 1   tpep_pickup_datetime   1500000 non-null  object 
 2   tpep_dropoff_datetime  1500000 non-null  object 
 3   passenger_count        1500000 non-null  int64  
 4   trip_distance          1500000 non-null  float64
 5   pickup_longitude       1500000 non-null  float64
 6   pickup_latitude        1500000 non-null  float64
 7   RatecodeID             1500000 non-null  int64  
 8   store_and_fwd_flag     1500000 non-null  object 
 9   dropoff_longitude      1500000 non-null  float64
 10  dropoff_latitude       1500000 non-null  float64
 11  payment_type           1500000 non-null  int64  
 12  fare_amount            1500000 non-null  float64
 13  extra                  1500000 non-null  float64
 14  mta_tax           

In [8]:
print("Valores nulos por coluna:")
nulos = df.isnull().sum()
nulos_percentual = (nulos / len(df) * 100).round(2)

resumo_nulos = pd.DataFrame({
    'Nulos': nulos,
    '% do total': nulos_percentual
})

print(resumo_nulos[resumo_nulos['Nulos'] > 0])

Valores nulos por coluna:
Empty DataFrame
Columns: [Nulos, % do total]
Index: []


In [9]:
colunas_criticas = ['passenger_count', 'trip_distance', 'fare_amount', 
                    'tip_amount', 'tolls_amount', 'total_amount',
                    'pickup_longitude', 'pickup_latitude',
                    'dropoff_longitude', 'dropoff_latitude']

df[colunas_criticas].describe().round(2)

,passenger_count,trip_distance,fare_amount,tip_amount,tolls_amount,total_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,1500000.00,1500000.00,1500000.00,1500000.00,1500000.00,1500000.00,1500000.00,1500000.00,1500000.00,1500000.00
mean,1.66,2.99,12.68,1.77,0.30,15.88,-72.84,40.13,-72.90,40.16
std,1.32,107.10,137.88,2.51,1.55,138.11,9.07,5.00,8.83,4.86
min,0.00,0.00,-140.00,-23.00,-5.54,-140.30,-121.93,0.00,-121.93,0.00
25%,1.00,1.00,6.50,0.00,0.00,8.30,-73.99,40.74,-73.99,40.73
50%,1.00,1.69,9.00,1.32,0.00,11.75,-73.98,40.75,-73.98,40.75
75%,2.00,3.10,14.00,2.35,0.00,17.30,-73.97,40.77,-73.96,40.77
max,9.00,131091.40,126348.88,299.75,559.54,126366.58,0.00,46.61,0.00,44.29


In [10]:
categoricas = ['payment_type', 'RatecodeID', 'store_and_fwd_flag', 'mes_origem']

for coluna in categoricas:
    print(f"\n{coluna}:", df[coluna].value_counts())


payment_type: payment_type
1    999315
2    493330
3      5423
4      1932
Name: count, dtype: int64

RatecodeID: RatecodeID
1     1462064
2       30247
5        4496
3        2482
4         653
99         42
6          16
Name: count, dtype: int64

store_and_fwd_flag: store_and_fwd_flag
N    1491502
Y       8498
Name: count, dtype: int64

mes_origem: mes_origem
01    500000
02    500000
03    500000
Name: count, dtype: int64


## Observações do Data Profiling

**Valores nulos:** Nenhum valor nulo identificado em nenhuma coluna, resultado incomum em dados operacionais reais.

**payment_type:** 4 categorias identificadas. Pela documentação oficial do TLC, apenas os tipos 1 (cartão de crédito) e 2 (dinheiro) representam corridas reais. Os tipos 3 (sem cobrança) e 4 (disputa de tarifa) totalizam 7.503 registros que serão removidos na etapa de limpeza, pois não representam corridas comerciais concluídas de forma convencional. Como o objetivo neste caso é analisar o padrão operacional e financeiro das corridas regulares de táxi em Nova York, manter esses registros introduziria ruído sistemático nos principais indicadores de negócio.

**RatecodeID:** O código 99 aparece 37 vezes, mas não consta na documentação oficial, cujas categorias válidas vão de 1 a 6, sendo:
1- Tarifa padrão
2- JFK
3- Newark
4- Nassau/Westchester
5- Tarifa negociada
6- Corrida em grupo
Por não ter definição oficial e representar menos de 0,003% da amostra, esses 37 registros serão removidos.

**store_and_fwd_flag:** Essa coluna indica se o registro da corrida foi armazenado localmente no taxímetro antes de ser enviado ao servidor central, o que ocorre quando o veículo está em área sem cobertura de rede. Como 99,4% dos registros são N (envio direto) e os 8.649 registros Y não indicam problema de qualidade nos dados, a coluna não agrega valor analítico e será descartada para simplificar o dataset.

**passenger_count:** Mínimo de zero: corridas sem passageiros são operacionalmente impossíveis e indicam falha no registro do taxímetro, portanto serão removidas. Máximo de 9 está dentro do limite legal para vans de taxi em NYC.

**trip_distance:** Mínimo de 0 milhas (corrida que não saiu do lugar) e máximo de 284.000 milhas (equivalente a 11 voltas ao redor do mundo). Ambos são erros de registro, e será aplicado um filtro entre 0,1 e 100 milhas, cobringo o range operacional real de corridas na cidade.

**fare_amount:** Valores negativos são fisicamente impossíveis em qualquer sistema de cobrança, e valores acima de 4.000 são extremamente suspeitos e configuram outliers. Serão mantidos apenas registros com tarifa entre 2,50 (valor mínimo oficial) e 500.

**tip_amount / tolls_amount:** Gorjetas e pedágios negativos são impossíveis, indicam estornos ou erros de sistema registrados de forma incorreta, portanto serão filtrados valores menores que zero.

**Coordenadas geográficas:** Nova York está localizada entre as longitudes -74,3 e -73,7 e as latitudes 40,4 e 40,9. Valores de longitude 0 (Meridiano de Greenwich) e latitude 0 (oceano Atlântico) confirmam falhas de GPS. Todos os registros com coordenadas fora do bounding box da cidade serão removidos.

## Problemas identificados para tratamento

**Conversão de datas:** As colunas tpep_pickup_datetime e tpep_dropoff_datetime estão armazenadas como texto, e precisam ser convertidas para o tipo datetime para permitir análises temporais como hora do dia, dia da semana e duração da corrida.

**Remoção de registros inválidos:** payment_type 3 e 4, e RatecodeID 99 serão removidos pelos motivos descritos acima.

**Descarte de colunas sem valor analítico:** store_and_fwd_flag será removida por não contribuir com nenhuma análise de negócio relevante.

**Valores impossíveis em colunas numéricas:** A análise descritiva apontou a presença provável de valores que ferem restrições físicas e operacionais básicas, como tarifas negativas, distâncias zero ou negativas, corridas sem nenhum passageiro e gorjetas negativas. Cada um desses será investigado, quantificado e tratado com critério na etapa de limpeza, preservando o máximo de registros válidos possível.

## Limpeza e tratamento dos dados

Com base nos problemas identificados, esta etapa aplica um conjunto de transformações e filtros para garantir que apenas registros válidos e consistentes sejam utilizados nas análises. Cada decisão de limpeza é documentada e justificada.

In [11]:
# converter datas de texto para datetime

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

print(f"Tipo pickup: {df['tpep_pickup_datetime'].dtype}")
print(f"Tipo dropoff: {df['tpep_dropoff_datetime'].dtype}")

Tipo pickup: datetime64[ns]
Tipo dropoff: datetime64[ns]


In [12]:
# calcular duração da viagem em minutos

df['duracao_min'] = (
    (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
)

print(f"Duração mínima: {df['duracao_min'].min():.1f} min")
print(f"Duração máxima: {df['duracao_min'].max():.1f} min")
print(f"Duração média: {df['duracao_min'].mean():.1f} min")

Duração mínima: -50520.2 min
Duração máxima: 117708.0 min
Duração média: 15.4 min


In [13]:
# componentes temporais

df['hora'] = df['tpep_pickup_datetime'].dt.hour
df['dia_semana'] = df['tpep_pickup_datetime'].dt.day_name()
df['dia_semana_num'] = df['tpep_pickup_datetime'].dt.dayofweek  # Segunda=0, Domingo=6
df['mes'] = df['tpep_pickup_datetime'].dt.month
df['dia_mes'] = df['tpep_pickup_datetime'].dt.day

In [14]:
# descartar colunas sem valor analítico

colunas_descartar = ['store_and_fwd_flag']
df = df.drop(columns=colunas_descartar)

In [15]:
# registrar tamanho original antes da limpeza

registros_originais = len(df)
print(f"Registros antes da limpeza: {registros_originais:,}")

Registros antes da limpeza: 1,500,000


In [17]:
# aplicação dos filtros

df_limpo = df[
    # passageiros: mínimo 1, máximo 9
    (df['passenger_count'] >= 1) & (df['passenger_count'] <= 9) &

    # distância: mínimo 0.1 milhas, máximo 100 milhas
    (df['trip_distance'] >= 0.1) & (df['trip_distance'] <= 100) &

    # valor da corrida: mínimo $2.50, máximo $500
    (df['fare_amount'] >= 2.50) & (df['fare_amount'] <= 500) &

    # gorjeta: sem valores negativos
    (df['tip_amount'] >= 0) &

    # pedágios: sem valores negativos
    (df['tolls_amount'] >= 0) &

    # valor total: mínimo $2.50, máximo $500
    (df['total_amount'] >= 2.50) & (df['total_amount'] <= 500) &

    # duração: mínimo 1 minuto, máximo 180 minutos
    (df['duracao_min'] >= 1) & (df['duracao_min'] <= 180) &

    # payment type: apenas formas de pagamento válidas (1 ou 2)
    (df['payment_type'] == 1) | (df['payment_type'] == 2) &

    # RatecodeID: apenas códigos válidos (1 a 6)
    (df['RatecodeID'] >= 1) & (df['RatecodeID'] <= 6) &

    # coordenadas: dentro dos limites aproximados de NYC
    (df['pickup_longitude'] >= -74.3) & (df['pickup_longitude'] <= -73.7) &
    (df['pickup_latitude'] >= 40.4) & (df['pickup_latitude'] <= 40.9) &
    (df['dropoff_longitude'] >= -74.3) & (df['dropoff_longitude'] <= -73.7) &
    (df['dropoff_latitude'] >= 40.4) & (df['dropoff_latitude'] <= 40.9) &

    # filtrar apenas os meses de janeiro a março de 2016
    (df['mes_origem'].isin(['01', '02', '03']))
]

print(f"Registros originais: {registros_originais:,}")
print(f"Registros após limpeza: {len(df_limpo):,}")
print(f"Registros removidos: {registros_originais - len(df_limpo):,}")
print(f"Taxa de aproveitamento: {len(df_limpo)/registros_originais*100:.1f}%")

Registros originais: 1,500,000
Registros após limpeza: 1,476,362
Registros removidos: 23,638
Taxa de aproveitamento: 98.4%


In [18]:
# detalhamento individual dos filtros

filtros = {
    'passenger_count inválido'    : df[(df['passenger_count'] < 1) | (df['passenger_count'] > 6)],
    'trip_distance inválida'      : df[(df['trip_distance'] <= 0.1) | (df['trip_distance'] > 100)],
    'fare_amount inválido'        : df[(df['fare_amount'] < 2.50) | (df['fare_amount'] > 500)],
    'tip_amount negativo'         : df[df['tip_amount'] < 0],
    'tolls_amount negativo'       : df[df['tolls_amount'] < 0],
    'total_amount inválido'       : df[(df['total_amount'] < 2.50) | (df['total_amount'] > 500)],
    'duração inválida'            : df[(df['duracao_min'] < 1) | (df['duracao_min'] > 180)],
    'payment_type inválido'       : df[~df['payment_type'].isin([1, 2])],
    'RatecodeID inválido'         : df[~df['RatecodeID'].isin([1, 2, 3, 4, 5, 6])],
    'coordenadas fora de NYC'     : df[
        ~df['pickup_longitude'].between(-74.3, -73.7) |
        ~df['pickup_latitude'].between(40.4, 40.9) |
        ~df['dropoff_longitude'].between(-74.3, -73.7) |
        ~df['dropoff_latitude'].between(40.4, 40.9)
    ],
    'data fora do período'        : df[
        (df['tpep_pickup_datetime'].dt.year != 2016) |
        (~df['tpep_pickup_datetime'].dt.month.isin([1, 2, 3]))
    ],
}

print("Impacto individual de cada filtro:")
print(f"{'Filtro':<35} {'Registros removidos':>20} {'% do total':>12}")
print("-" * 70)
for nome, subset in filtros.items():
    qtd = len(subset)
    pct = qtd / registros_originais * 100
    print(f"{nome:<35} {qtd:>20,} {pct:>11.2f}%")

Impacto individual de cada filtro:
Filtro                               Registros removidos   % do total
----------------------------------------------------------------------
passenger_count inválido                              80        0.01%
trip_distance inválida                            12,508        0.83%
fare_amount inválido                               1,293        0.09%
tip_amount negativo                                   13        0.00%
tolls_amount negativo                                  3        0.00%
total_amount inválido                              1,164        0.08%
duração inválida                                  14,475        0.97%
payment_type inválido                              7,355        0.49%
RatecodeID inválido                                   42        0.00%
coordenadas fora de NYC                           26,693        1.78%
data fora do período                                   0        0.00%


In [19]:
# salvar dataset limpo

df_limpo.to_parquet('../data/nyc_taxi_limpo.parquet', index=False)

print(f"Registros: {len(df_limpo):,}")
print(f"Colunas: {df_limpo.shape[1]}")

Registros: 1,476,362
Colunas: 25


## Análise exploratória

Depois da limpeza e validação do dataset, são respondidas as perguntas de negócio definidas no início do projeto.

In [22]:
# carregar dataset limpo para análise

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = (13, 5)
plt.rcParams['font.size'] = 12

# carregar dataset limpo
df = pd.read_parquet('../data/nyc_taxi_limpo.parquet')

print(f"Dataset limpo carregado")

df.head()

Dataset limpo carregado


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,...,tolls_amount,improvement_surcharge,total_amount,mes_origem,duracao_min,hora,dia_semana,dia_semana_num,mes,dia_mes
0,2,2016-01-01 00:00:00,2016-01-01 00:26:45,2,7.45,-73.994057,40.719990,1,-73.966362,40.789871,...,0.0,0.3,27.3,01,26.750000,0,Friday,4,1,1
1,1,2016-01-01 00:00:09,2016-01-01 00:07:18,2,1.20,-73.963913,40.712173,1,-73.951332,40.712200,...,0.0,0.3,8.3,01,7.150000,0,Friday,4,1,1
2,2,2016-01-01 00:00:18,2016-01-01 00:05:44,5,1.17,-73.963058,40.775017,1,-73.951744,40.778801,...,0.0,0.3,7.3,01,5.433333,0,Friday,4,1,1
3,1,2016-01-01 00:00:52,2016-01-01 00:08:00,3,1.80,-73.983871,40.742409,1,-74.002365,40.726349,...,0.0,0.3,9.3,01,7.133333,0,Friday,4,1,1
4,1,2016-01-01 00:01:13,2016-01-01 00:22:09,1,2.50,-73.994255,40.731995,1,-73.978638,40.751389,...,0.0,0.3,15.8,01,20.933333,0,Friday,4,1,1
